#### Install and import Python libraries

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
import sys

root_dir = ".."
sys.path.append(os.path.join(root_dir, "src"))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader

from mot.data.data_track import MOT16Sequences
from mot.data.data_obj_detect import MOT16ObjDetect
from mot.models.object_detector import FRCNN_FPN
from mot.tracker.base import Tracker, BaseReIDTracker
from mot.eval import get_mot_accum,evaluate_obj_detect, evaluate_mot_accums
from mot.visualize import plot_sequence
from mot.utils import ltrb_to_ltwh

from scipy.optimize import linear_sum_assignment as linear_assignment
from market.models import build_model
from mot.tracker.base import Tracker

import motmetrics as mm
mm.lap.default_solver = 'lap'
%matplotlib inline


## Example sequences

In [ ]:
seq_name = "MOT16-02"
data_dir = os.path.join(root_dir, "data/MOT16")
sequences = MOT16Sequences(seq_name, data_dir, load_seg=True)  # Get a coffee.

for seq in sequences:
    for i, frame in enumerate(seq):
        img = frame["img"]

        dpi = 96
        fig, ax = plt.subplots(1, dpi=dpi)

        img = img.mul(255).permute(1, 2, 0).byte().numpy()
        width, height, _ = img.shape

        ax.imshow(img, cmap="gray")
        fig.set_size_inches(width / dpi, height / dpi)

        if "gt" in frame:
            gt = frame["gt"]
            for gt_id, box in gt.items():
                rect = plt.Rectangle(
                    (box[0], box[1]),
                    box[2] - box[0],
                    box[3] - box[1],
                    fill=False,
                    linewidth=1.0,
                )
                ax.add_patch(rect)

        plt.axis("off")
        plt.show()
        break


# Object detector

The object detector is a [Faster R-CNN](https://arxiv.org/abs/1506.01497) with a Resnet50 feature extractor. We trained the native PyTorch implementation of Faster-RCNN. For more information check out the corresponding PyTorch [webpage](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html).


## Configuration

In [ ]:
obj_detect_model_file = os.path.join(root_dir, "models/faster_rcnn_fpn.model")
obj_detect_nms_thresh = 0.3


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# object detector
obj_detect = FRCNN_FPN(num_classes=2, nms_thresh=obj_detect_nms_thresh)
obj_detect_state_dict = torch.load(
    obj_detect_model_file, map_location=lambda storage, loc: storage
)
obj_detect.load_state_dict(obj_detect_state_dict)
obj_detect.eval()
obj_detect = obj_detect.to(device)


# Multi-object tracking

## Configuration

In [ ]:
seed = 12345
seq_name = "MOT16-reid"  # We recommend to use this subset.
data_dir = os.path.join(root_dir, "data/MOT16")
output_dir = os.path.join(root_dir, "output")


## Setup

In [ ]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

# dataset
sequences = MOT16Sequences(seq_name, data_dir)


# Reid + Hungarian algo+ cosine dist

In this part, we combine the ReID model with the tracker to compute appearance-based matching.

Since we're using a very simple ReID model, tracking only based on apperance is very difficult.
For all pairs of tracks and detections, the distance is set to `_UNMATCHED_COST`, when the IoU value is below 0.5. For all other cases, the costs are based on the cosine distance. 
The cosine distance should be between -1 and 1.

## reid model

In [ ]:
model_path = root_dir + "/models/resnet34_reid_market.model"

reid_model = build_model("resnet34", num_classes=751, loss="triplet", pretrained=True)
reid_market_state_dict = torch.load(
    model_path, map_location=lambda storage, loc: storage
)
reid_model.load_state_dict(reid_market_state_dict)


In [ ]:
model_path = root_dir + "/models/resnet50_reid.pth"

reid_model = build_model("resnet34", 751, loss="softmax", pretrained=True)
reid_ckpt = torch.load(model_path)
reid_model.load_state_dict(reid_ckpt)


In [ ]:
reid_model = reid_model.cuda()


In [ ]:
def cosine_distance(input1, input2):
    """Computes cosine distance.
    Args:
        input1 (torch.Tensor): 2-D feature matrix (m x feat).
        input2 (torch.Tensor): 2-D feature matrix (n x feat).
    Returns:
        torch.Tensor: distance matrix (m x n).
    """

    # Given that cos_sim(u, v) = dot(u, v) / (norm(u) * norm(v))
    #                          = dot(u / norm(u), v / norm(v))
    # We fist normalize the rows, before computing their dot products via transposition:
    norm1 = input1.norm(dim=1)[:, None]
    norm2 = input2.norm(dim=1)[:, None]
    input1_norm = input1 / norm1
    input2_norm = input2 / norm2
    cosine_similarity = torch.mm(input1_norm, input2_norm.t())
    distmat = 1 - cosine_similarity
    return distmat


In [ ]:
_UNMATCHED_COST = 255.0


class ReIDHungarianTracker(BaseReIDTracker):
    def data_association(self, boxes, scores, frame):
        crops = self.get_crop_from_boxes(boxes, frame)
        pred_features = self.compute_reid_features(reid_model, crops).cpu().clone()

        if self.tracks:
            track_ids = [t.id for t in self.tracks]
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0)
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0)

            # This will use your similarity measure. Please use cosine_distance!
            distance = self.compute_distance_matrix(
                track_features,
                pred_features,
                track_boxes,
                boxes,
                metric_fn=cosine_distance,
            )

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)

            # row_idx and col_idx are indices into track_boxes and boxes.
            # row_idx[i] and col_idx[i] define a match.
            # distance[row_idx[i], col_idx[i]] define the cost for that matching.

            remove_track_ids = []
            seen_track_ids = []
            seen_box_idx = []
            # Update existing tracks and remove unmatched tracks.

            for track_idx, box_idx in zip(row_idx, col_idx):
                costs = distance[track_idx, box_idx]
                internal_track_id = track_ids[track_idx]
                seen_track_ids.append(internal_track_id)
                # If the costs are equal to _UNMATCHED_COST, it's not a match.
                if costs == _UNMATCHED_COST:
                    remove_track_ids.append(internal_track_id)
                else:
                    self.tracks[track_idx].box = boxes[box_idx]
                    self.tracks[track_idx].add_feature(pred_features[box_idx])
                    seen_box_idx.append(box_idx)

            unseen_track_ids = set(track_ids) - set(seen_track_ids)
            remove_track_ids.extend(list(unseen_track_ids))
            self.tracks = [t for t in self.tracks if t.id not in remove_track_ids]

            # update the feature of a track by using add_feature:
            # self.tracks[my_track_id].add_feature(pred_features[my_feat_index])
            # use the mean feature from the last 10 frames for ReID.

            new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
            new_boxes = [boxes[i] for i in new_boxes_idx]
            new_scores = [scores[i] for i in new_boxes_idx]
            new_features = [pred_features[i] for i in new_boxes_idx]
            self.add(new_boxes, new_scores, new_features)
        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)


In [ ]:
print(ReIDHungarianTracker.__mro__)

### Run tracker 


In [ ]:
# Old Tracker
# tracker = BaseTrackerIoU(obj_detect)
# tracker = HungarianIoUTracker(obj_de|tect)

tracker = ReIDHungarianTracker(obj_detect)


In [ ]:
time_total = 0
mot_accums = []
results_seq = {}
for seq in sequences:
    tracker.reset()
    now = time.time()

    print(f"Tracking: {seq}")

    data_loader = DataLoader(seq, batch_size=1, shuffle=False)

    for frame in tqdm(data_loader):
        tracker.step(frame)
    results = tracker.get_results()
    results_seq[str(seq)] = results

    if seq.no_gt:
        print(f"No GT evaluation data available.")
    else:
        mot_accums.append(get_mot_accum(results, seq))

    time_total += time.time() - now

    print(f"Tracks found: {len(results)}")
    print(f"Runtime for {seq}: {time.time() - now:.1f} s.")

    # seq.write_results(results, os.path.join(output_dir))

print(f"Runtime for all sequences: {time_total:.1f} s.")
if mot_accums:
    evaluate_mot_accums(
        mot_accums, [str(s) for s in sequences if not s.no_gt], generate_overall=True
    )


## Baseline Tracker Results

            IDF1   IDP   IDR   Rcll  Prcn  GT  MT  PT ML  FP   FN IDs   FM  MOTA  MOTP
    MOT16-02 32.2% 49.8% 23.8% 30.8% 64.4%  62  5  22 35 3170 12858  52   93 13.5% 0.086
    MOT16-05 47.7% 53.9% 42.8% 57.8% 72.7% 133 39  64 30 1502  2917  87  103 34.9% 0.144
    MOT16-09 43.0% 48.8% 38.4% 51.9% 66.1%  26  7  14  5 1420  2559  39   66 24.5% 0.107
    MOT16-11 49.0% 54.1% 44.8% 55.8% 67.5%  75 15  32 28 2542  4166  20   39 28.7% 0.080
    OVERALL  41.0% 51.8% 33.9% 44.1% 67.3% 296 66 132 98 8634 22500 198  301 22.2% 0.101

## Hungarian Tracker Results

             IDF1    IDP  IDR  Rcll  Prcn   GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP 
    MOT16-02 39.1% 55.5% 30.2% 52.3% 96.2%  62  11  38 13  383  8870 246  215 48.9% 0.096 
    MOT16-05 55.1% 65.2% 47.7% 68.8% 94.2% 133  55  66 12  295  2158 199  155 61.7% 0.143  
    MOT16-09 50.2% 62.0% 42.1% 66.4% 97.8%  26  13  12  1   80  1789  76   78 63.5% 0.083  
    MOT16-11 60.4% 66.6% 55.3% 80.2% 96.6%  75  42  26  7  266  1868  99   86 76.3% 0.083  
    OVERALL  49.0% 61.5% 40.6% 63.5% 96.2% 296 121 142 33 1024 14685 620  534 59.4% 0.099 

# ReidHungarianIoU Tracker results

              IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP IDt IDa IDm
    MOT16-02 41.0% 58.2% 31.6% 52.3% 96.2%  62  11  38 13  383  8870 334  221 48.4% 0.095 173 154  10
    MOT16-05 55.8% 66.1% 48.3% 68.8% 94.2% 133  56  65 12  295  2158 218  150 61.4% 0.142  96 144  25
    MOT16-09 52.4% 64.7% 44.0% 66.4% 97.8%  26  12  13  1   80  1789  80   79 63.4% 0.083  27  58   5
    MOT16-11 62.2% 68.6% 56.9% 80.2% 96.6%  75  42  26  7  266  1868 112   86 76.2% 0.083  41  77  11
    OVERALL  50.6% 63.7% 42.1% 63.5% 96.2% 296 121 142 33 1024 14685 744  536 59.1% 0.099 337 433  51

## Visualize tracking results

In [ ]:
plot_sequence(
    results_seq["MOT16-02"],
    [s for s in sequences if str(s) == "MOT16-02"][0],
    first_n_frames=3,
)
